<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/note_books/lgbm_all_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import numpy as np
import pandas as pd
import shutil
import scipy.stats as stats
from pathlib import Path
import glob
from tqdm import tqdm


from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager

from sklearn.preprocessing import LabelEncoder

In [2]:
N_SPLITS = 10
SEED = 618


In [3]:
LOG_PATH = Path("./log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)


In [4]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


In [5]:
def set_seed(seed=527):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, x, y):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2))
    return intermediate.sum()/xhat.shape[0]


def score_log(df: pd.DataFrame, num_files: int, nam_file: str, data_shape: tuple, n_fold: int, seed: int, mpe: float):
    score_dict = {'n_files': num_files, 'file_name': nam_file, 'shape': data_shape, 'fold': n_fold, 'seed': seed, 'score': mpe}
    # noinspection PyTypeChecker
    df = pd.concat([df, pd.DataFrame.from_dict([score_dict])])
    df.to_csv(LOG_PATH / f"log_score.csv", index=False)
    return df

# **特徴量関数**

In [64]:
#train用

def get_numerical_features_train(input_df):
    input_df_ = input_df['path']
    output_df = input_df.drop(['Unnamed: 0.1', 'Unnamed: 0.1.1',
                               'beacon_timestamp', 'beacon_uuid', 'beacon_major_id', 'beacon_minor_id', 'beacon_mac_addr', 'beacon_timestamp2',
                               'accelerometer_timestamp', 'accelerometer_uncalibrated_timestamp', 'gyroscope_timestamp', 'gyroscope_uncalibrated_timestamp',
                               'magnetic_field_timestamp', 'magnetic_field_uncalibrated_timestamp', 'rotation_vector_timestamp', 'timestamp', 'path'], axis=1)
    output_df['path'] = input_df_
    return output_df


def get_sum_sensor_features(input_df):
    output_df = pd.DataFrame()
    output_df['sum_sensor_x'] = input_df['accelerometer_x'] + input_df['accelerometer_uncalibrated_x'] + input_df['accelerometer_uncalibrated_x2'] + input_df['gyroscope_x'] + input_df['gyroscope_uncalibrated_x'] + input_df['accelerometer_uncalibrated_x'] + input_df['gyroscope_uncalibrated_x2'] + input_df['magnetic_field_x'] + input_df['magnetic_field_uncalibrated_x'] + input_df['magnetic_field_uncalibrated_x2'] + input_df['rotation_vector_x']
    output_df['sum_sensor_y'] = input_df['accelerometer_y'] + input_df['accelerometer_uncalibrated_y'] + input_df['accelerometer_uncalibrated_y2'] + input_df['gyroscope_y'] + input_df['gyroscope_uncalibrated_y'] + input_df['accelerometer_uncalibrated_y'] + input_df['gyroscope_uncalibrated_y2'] + input_df['magnetic_field_y'] + input_df['magnetic_field_uncalibrated_y'] + input_df['magnetic_field_uncalibrated_y2'] + input_df['rotation_vector_y']
    output_df['sum_sensor_z'] = input_df['accelerometer_x'] + input_df['accelerometer_uncalibrated_x'] + input_df['accelerometer_uncalibrated_x2'] + input_df['gyroscope_z'] + input_df['gyroscope_uncalibrated_z'] + input_df['accelerometer_uncalibrated_z'] + input_df['gyroscope_uncalibrated_z2'] + input_df['magnetic_field_z'] + input_df['magnetic_field_uncalibrated_z'] + input_df['magnetic_field_uncalibrated_z2'] + input_df['rotation_vector_z']
    return output_df


def get_le_features(input_df):
    _input_df = input_df.astype(str)
    cat_cols = ['beacon_uuid', 'beacon_major_id', 'beacon_minor_id', 'beacon_mac_addr']
    output_df = pd.DataFrame()
    for c in cat_cols:
        le = LabelEncoder()
        le.fit(_input_df[c])
        output_df["LE_"+c] = le.transform(_input_df[c])
    return output_df





In [78]:
#test用
def get_numerical_features_test(input_df):
    input_df_ = input_df['site_path_timestamp']
    output_df = input_df.drop(['Unnamed: 0.1', 'Unnamed: 0.1.1',
                               'beacon_timestamp', 'beacon_uuid', 'beacon_major_id', 'beacon_minor_id', 'beacon_mac_addr', 'beacon_timestamp2',
                               'accelerometer_timestamp', 'accelerometer_uncalibrated_timestamp', 'gyroscope_timestamp', 'gyroscope_uncalibrated_timestamp',
                               'magnetic_field_timestamp', 'magnetic_field_uncalibrated_timestamp', 'rotation_vector_timestamp', 'timestamp','site_path_timestamp'
                               ], axis=1)
    output_df['site_path_timestamp'] = input_df_
    return output_df


In [79]:
def get_process_funcs_train():
    funcs = [
        get_sum_sensor_features,
        get_le_features,
        get_numerical_features_train
    ]
    return funcs

def get_process_funcs_test():
    funcs = [
        get_sum_sensor_features,
        get_le_features,
        get_numerical_features_test
    ]
    return funcs

def to_feature(input_df, funcs):
    output_df = pd.DataFrame()
    for func in tqdm(funcs, total=len(funcs)):
        _df = func(input_df)
        assert len(_df) == len(input_df), func.__name__
        output_df = pd.concat([output_df, _df], axis=1)

    return output_df

In [80]:
feature_dir = "/content/drive/MyDrive/all_data_floor"
train_files = sorted(glob.glob(os.path.join(f'{feature_dir}','*_train.csv')))
test_files = sorted(glob.glob(os.path.join(f'{feature_dir}','*_test.csv')))
subm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv', index_col=0)
floor_accurate = pd.read_csv('/content/drive/MyDrive/submission_floor_accurate.csv').drop(['x', 'y'], axis=1)

In [81]:
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

In [84]:
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0).fillna(0)
    test_data = pd.read_csv(test_files[n_files], index_col=0).fillna(0)

    # all featrues
    process_funcs_train = get_process_funcs_train()
    process_funcs_test = get_process_funcs_test()
    data = to_feature(data, process_funcs_train).rename(columns={'f': 'floor'})
    test_data = to_feature(test_data, process_funcs_test)

    print(data.shape)
    print(test_data.shape)

    oof_x, oof_y = np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-3])):
        X_train = data.iloc[trn_idx, :-3]
        y_trainx = data.iloc[trn_idx, -3]
        y_trainy = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-3]
        y_validx = data.iloc[val_idx, -3]
        y_validy = data.iloc[val_idx, -2]
        
        modelx = lgb.LGBMRegressor(**lgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=False,
                       early_stopping_rounds=20
                       )
            
        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)

        preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], y_validx.to_numpy(), y_validy.to_numpy())
        print(f"fold {fold}: mean position error {score}")
        score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, fold, SEED, score)

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)
    score_df = score_log(score_df, n_files, os.path.basename(file), data.shape, 999, SEED, score)


    test_preds = pd.DataFrame()
    test_preds['floor'] = test_data['floor'].values
    test_preds['x'] = pd.DataFrame(preds_x).T
    test_preds['y'] = pd.DataFrame(preds_y).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    predictions.append(test_preds)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3402,3403,3404,3408) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)




  0%|          | 0/3 [00:00<?, ?it/s]



 67%|██████▋   | 2/3 [00:20<00:10, 10.49s/it]



100%|██████████| 3/3 [00:21<00:00,  7.04s/it]




  0%|          | 0/3 [00:00<?, ?it/s]



100%|██████████| 3/3 [00:00<00:00,  4.23it/s]


(9296, 3441)
(299, 3439)


[5.3GB(-0.0GB): 24.034sec] fit X
[5.3GB(+0.0GB): 21.220sec] fit Y


fold 0: mean position error 2.6808782826916806


[5.3GB(+0.0GB): 12.839sec] fit X
[5.3GB(+0.0GB): 24.046sec] fit Y


...

fold 7: mean position error 3.2731159761358497


[9.8GB(+0.0GB): 42.283sec] fit X
[9.8GB(+0.0GB): 51.530sec] fit Y


fold 8: mean position error 3.422965910474774


[9.8GB(+0.0GB): 40.205sec] fit X
[9.8GB(+0.0GB): 54.662sec] fit Y


fold 9: mean position error 3.1664771687548265
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+
file #23, shape=(15655, 4908), name=5dc8cea7659e181adb076a3f_train.csv
mean position error 3.285773280371117
*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+*+


In [85]:
all_preds = pd.concat(predictions)

all_preds = all_preds.reindex(subm.index)
all_preds['floor'] = floor_accurate['floor'].values
all_preds.to_csv('submission_lgbm_wifi_0_016.csv')

In [86]:
shutil.move('submission_lgbm_wifi_0_016.csv', '/content/drive/MyDrive')

'/content/drive/MyDrive/submission_lgbm_wifi_0_016.csv'

In [97]:
all_preds

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,99.723513,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,102.805032,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,106.057743,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,105.801775,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,112.147585,0
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,5,88.326606,5
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,5,90.664590,5
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,5,103.712496,5


In [96]:
test_preds

,floor,x,y
site_path_timestamp,,,
5dc8cea7659e181adb076a3f_b3ddea2aa88e2015223b75a8_0000000000006,118.185898,99.382318,6
5dc8cea7659e181adb076a3f_a850f1538a738b74d58cb9bf_0000000000006,112.301320,108.798547,6
5dc8cea7659e181adb076a3f_c2ae6f835a79f6fd53600449_0000000000006,50.708239,72.925214,5
5dc8cea7659e181adb076a3f_8ccd3fc53ad2e63651de4376_0000000000007,47.056859,66.772858,2
5dc8cea7659e181adb076a3f_6e1c6452fd95c2d3e6785f36_0000000000007,176.604809,111.277064,3
...,...,...,...
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000181673,182.195726,80.078049,-1
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000184177,183.757525,81.780066,-1
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000195142,185.511752,80.449858,-1


In [99]:
all_preds = pd.concat(predictions)

In [112]:
all_preds = all_preds.rename(columns={'floor':'x', 'x':'y', 'y':'floor'})

In [113]:
preds_xy = all_preds.loc[:, ['x', 'y']]


In [122]:
a = pd.read_csv('/content/drive/MyDrive/submission_lstm_in_floor_07_before_post.csv')
a

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,88.987900,105.374695
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,86.266991,107.974129
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,84.922188,107.128471
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.189232,112.489365
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,91.785637,112.877716
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,210.201599,94.858597
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,208.465881,95.657104
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,203.611420,103.190163
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,200.114624,113.268608


In [115]:
all_preds.drop('floor', axis=1)

,x,y
site_path_timestamp,,
5a0546857ecc773753327266_ed77f28aeaf89b317bc380fa_0000000000001,92.259521,108.540651
5a0546857ecc773753327266_dd4cbd69218f610f27cf33c8_0000000000007,98.761775,85.706924
5a0546857ecc773753327266_bd0f2c8626679895f3338ca5_0000000000007,117.330338,169.415457
5a0546857ecc773753327266_ffcd9524c80c0fa5bb859eaf_0000000000007,129.568878,173.558543
5a0546857ecc773753327266_bd5921cd1008382e2f537b53_0000000000007,219.973290,161.484395
...,...,...
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000181673,182.195726,80.078049
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000184177,183.757525,81.780066
5dc8cea7659e181adb076a3f_e83a1c294b5d138339149fcb_0000000195142,185.511752,80.449858


In [116]:
all_preds['floor'] = floor_accurate['floor'].values

In [117]:
all_preds.to_csv('submission_lgbm_wifi_0_017.csv')
shutil.move('submission_lgbm_wifi_0_017.csv', '/content/drive/MyDrive')

In [119]:
shutil.move('submission_lgbm_wifi_0_017.csv', '/content/drive/MyDrive')

'/content/drive/MyDrive/submission_lgbm_wifi_0_017.csv'

In [127]:
all_preds = all_preds.reindex(subm.index)

In [128]:
all_preds

,x,y,floor
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,93.842121,99.723513,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,93.686649,102.805032,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,87.901474,106.057743,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,91.425563,105.801775,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,93.569127,112.147585,-1
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,216.540067,88.326606,-1
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,214.640180,90.664590,-1
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,203.259193,103.712496,-1


In [129]:
all_preds['floor'] = floor_accurate['floor'].values

In [130]:
all_preds

,x,y,floor
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,93.842121,99.723513,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,93.686649,102.805032,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,87.901474,106.057743,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,91.425563,105.801775,0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,93.569127,112.147585,0
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,216.540067,88.326606,5
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,214.640180,90.664590,5
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,203.259193,103.712496,5


In [125]:
subm

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0
...,...,...,...
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000082589,0,75.0,75.0
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000085758,0,75.0,75.0
5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f412_0000000090895,0,75.0,75.0


In [131]:
all_preds.to_csv('submission_lgbm_wifi_0_018.csv')
shutil.move('submission_lgbm_wifi_0_018.csv', '/content/drive/MyDrive')

'/content/drive/MyDrive/submission_lgbm_wifi_0_018.csv'